In [1]:
import os

In [2]:
%pwd

'/home/jafarid/code/yogaposes/research'

In [3]:
os.chdir('../')
%pwd

'/home/jafarid/code/yogaposes'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path 
    resnet_trained_model_path: Path
    resnet_updated_base_model_path: Path
    traning_data: Path
    params_augmentation: bool
    params_image_size: list 
    params_batch_size: int 
    params_epoches: int
    params_learning_rate: float

In [6]:
from yogaposes.constants import *
from yogaposes.utils.common import read_yaml, create_directories

In [8]:
class configurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_traning_config(self) -> TrainingConfig:
        model_training = self.config.model_training
        prepare_base_model = self.config.prepare_base_model
        training_data = os.path.join(self.config.data_ingestion.root_dir, 'yoga-poses-dataset')
        
        create_directories([model_training.root_dir])
        
        training_config = TrainingConfig(root_dir= model_training.resnet_trained_model_path, 
                                        resnet_trained_model_path= model_training.resnet_trained_model_path,
                                        resnet_updated_base_model_path= prepare_base_model.resnet_updated_base_model_path,
                                        traning_data = training_data,
                                        params_augmentation = self.params.AUGMENTATION,
                                        params_image_size = self.params.IMAGE_SIZE,
                                        params_batch_size= self.params.BATCH_SIZE,
                                        params_epoches = self.params.EPOCHS,
                                        params_learning_rate = self.params.LEARNING_RATE
                                        )
        
        return training_config

In [9]:
import torch
import torch.optim as optim
from torch import nn
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, CenterCrop
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [ ]:
class ModelTrainer(object):
    def __init__(self, config:TrainingConfig, loss_fn=None, optimizer=None):
        self.config = config
        self.model = self.load_model()
        self.loss_fn = loss_fn if loss_fn else nn.CrossEntropyLoss(reduction='mean')
        self.optimizer = optimizer if optimizer else optim.adam(self.model.parameters(), lr=self.config.params_learning_rate)
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model.to(self.device)
        
    def load_model(self):
        return torch.load(self.config.resnet_updated_base_model_path)
    
    def set_loaders(self):
        # image net statistics
        normalizer = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,0.224,0.225])
        
        composer = Compose([Resize(256), CenterCrop(224), ToTensor(), normalizer])
        
        train_data = ImageFolder(root=os.path.join(self.config.traning_data,'TRAIN'), transform=composer)
        val_data = ImageFolder(root=os.path.join(self.config.traning_data,'TEST'), transform=composer)
        
        train_loader = DataLoader(train_data, batch_size=self.config.params_batch_size, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=self.config.params_batch_size)
        
        return train_loader, val_loader
        